In [ ]:
#@title 📦 **Step 1: Setup & GPU Check** { display-mode: "form" }
#@markdown Run this cell first to check GPU and install dependencies

import os
import sys
import shutil
import random
import warnings
import zipfile
import glob
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

warnings.filterwarnings('ignore')

# Install dependencies
!pip install -q tqdm
from tqdm.auto import tqdm

# Check GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ GPU enabled: {gpus[0].name}")
else:
    print("❌ No GPU found! Go to Runtime → Change runtime type → GPU")

NUM_WORKERS = multiprocessing.cpu_count()
print(f"CPU cores: {NUM_WORKERS}")

In [ ]:
#@title 📤 **Step 2: Upload Dataset** { display-mode: "form" }
#@markdown Upload a ZIP file containing your dataset folders

from google.colab import files

# Setup directories
BASE_DIR = Path("/content/ml_models")
BASE_DIR.mkdir(parents=True, exist_ok=True)
LOCAL_CACHE = BASE_DIR / "combined_balanced_dataset"
OUTPUT_DIR = BASE_DIR

print("="*60)
print("📤 UPLOAD YOUR DATASET ZIP FILE")
print("="*60)
print("")
print("Your ZIP should contain folders like:")
print("  📂 Bigger CoLeaf DATASET/CoLeaf DATASET/")
print("  📂 Propossed_Data/")
print("  📂 Nitrogen deficiency/")
print("  📂 ThorCam_semiFiltered/")
print("  📂 POTASSIUM DEFICIENCY/")
print("")

uploaded = files.upload()

# Extract uploaded ZIP
for filename in uploaded.keys():
    print(f"\n📦 Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(str(BASE_DIR))
    os.remove(filename)
    print(f"✅ Extracted to {BASE_DIR}")

# Show contents
print("\n🔍 Extracted contents:")
for item in sorted(BASE_DIR.iterdir()):
    if item.is_dir():
        count = sum(1 for _ in item.rglob("*") if _.is_file())
        print(f"   📂 {item.name}/  ({count} files)")
    else:
        print(f"   📄 {item.name}")

In [ ]:
#@title ⚙️ **Step 3: Configuration** { display-mode: "form" }
#@markdown Configure training parameters

import numpy as np
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetV2S
from sklearn.model_selection import train_test_split
from collections import Counter

# Dataset paths
DATA_SOURCES = [
    BASE_DIR / "Bigger CoLeaf DATASET" / "CoLeaf DATASET",
    BASE_DIR / "Propossed_Data" / "Contrast_Stretching",
    BASE_DIR / "Propossed_Data" / "Histogram_Equalization",
    BASE_DIR / "Propossed_Data" / "Log_Transformation",
    BASE_DIR / "Nitrogen deficiency",
    BASE_DIR / "ThorCam_semiFiltered",
    BASE_DIR / "POTASSIUM DEFICIENCY",
]

# Class mapping
CLASS_MAPPING = {
    "healthy": "healthy", "control": "healthy", "-C": "healthy",
    "nitrogen-N": "nitrogen-N", "deficiency": "nitrogen-N", "N": "nitrogen-N",
    "phosphorus-P": "phosphorus-P", "-P": "phosphorus-P", "-P50": "phosphorus-P", "P": "phosphorus-P",
    "potasium-K": "potasium-K", "K": "potasium-K",
    "boron-B": "boron-B", "calcium-Ca": "calcium-Ca", "iron-Fe": "iron-Fe",
    "magnesium-Mg": "magnesium-Mg", "manganese-Mn": "manganese-Mn",
}

TARGET_CLASSES = ["healthy", "nitrogen-N", "phosphorus-P", "potasium-K"]
EXTRA_CLASSES = ["boron-B", "calcium-Ca", "iron-Fe", "magnesium-Mg", "manganese-Mn"]
ALL_CLASSES = TARGET_CLASSES + EXTRA_CLASSES

# Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32  # Larger for GPU
EPOCHS_PHASE1 = 30
EPOCHS_PHASE2 = 25
EPOCHS_PHASE3 = 20
VALIDATION_SPLIT = 0.15
TEST_SPLIT = 0.10
INITIAL_LR = 1e-3
FINE_TUNE_LR = 1e-4
LABEL_SMOOTHING = 0.1
DROPOUT_RATE = 0.4
EARLY_STOPPING_PATIENCE = 8
TARGET_SAMPLES_PER_CLASS = 500

print("✅ Configuration loaded")
print(f"   Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Classes: {len(ALL_CLASSES)}")
print(f"   Total epochs: {EPOCHS_PHASE1 + EPOCHS_PHASE2 + EPOCHS_PHASE3}")

In [ ]:
#@title 📂 **Step 4: Combine & Balance Datasets** { display-mode: "form" }
#@markdown Scan, combine, and balance all datasets

def copy_file(args):
    src, dest = args
    try:
        if not dest.exists():
            shutil.copy2(src, dest)
            return 1
    except:
        pass
    return 0

def collect_all_images():
    print("📁 Scanning all datasets...")
    class_images = {cls: [] for cls in ALL_CLASSES}
    
    for dataset_path in DATA_SOURCES:
        if not dataset_path.exists():
            print(f"   ⚠️ Not found: {dataset_path.name}")
            continue
        
        dataset_name = dataset_path.name
        print(f"   📂 Scanning {dataset_name}...")
        
        if "Nitrogen" in dataset_name or "nitrogen" in dataset_name:
            for split in ["train", "val", "test"]:
                split_dir = dataset_path / split
                if split_dir.exists():
                    for class_dir in split_dir.iterdir():
                        if class_dir.is_dir():
                            std_class = CLASS_MAPPING.get(class_dir.name)
                            if std_class and std_class in class_images:
                                for img in class_dir.glob("*"):
                                    if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                                        class_images[std_class].append(img)
        elif "ThorCam" in dataset_name:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std_class = CLASS_MAPPING.get(class_dir.name)
                    if std_class and std_class in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".tif"]:
                                class_images[std_class].append(img)
        elif "POTASSIUM" in dataset_name.upper():
            for img in dataset_path.glob("*"):
                if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                    class_images["potasium-K"].append(img)
        else:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std_class = CLASS_MAPPING.get(class_dir.name, class_dir.name)
                    if std_class and std_class in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp"]:
                                class_images[std_class].append(img)
    return class_images

def balance_classes(class_images):
    print(f"\n⚖️ Balancing classes (target: {TARGET_SAMPLES_PER_CLASS})...")
    balanced = {cls: list(imgs) for cls, imgs in class_images.items()}
    
    for cls in ALL_CLASSES:
        current = len(balanced[cls])
        if current == 0:
            print(f"   ⚠️ {cls}: No samples!")
            continue
        if current < TARGET_SAMPLES_PER_CLASS:
            needed = TARGET_SAMPLES_PER_CLASS - current
            balanced[cls].extend(random.choices(balanced[cls], k=needed))
            print(f"   📈 {cls}: {current} → {len(balanced[cls])}")
        elif current > TARGET_SAMPLES_PER_CLASS * 2:
            balanced[cls] = random.sample(balanced[cls], TARGET_SAMPLES_PER_CLASS)
            print(f"   📉 {cls}: {current} → {len(balanced[cls])}")
        else:
            print(f"   ✓ {cls}: {current}")
    return balanced

# Run
if LOCAL_CACHE.exists():
    shutil.rmtree(LOCAL_CACHE)

class_images = collect_all_images()

print("\n📊 Raw dataset statistics:")
total = sum(len(imgs) for imgs in class_images.values())
for cls in ALL_CLASSES:
    print(f"   {cls:15s}: {len(class_images[cls]):5d}")
print(f"   {'TOTAL':15s}: {total:5d}")

balanced_images = balance_classes(class_images)

for cls in ALL_CLASSES:
    (LOCAL_CACHE / cls).mkdir(parents=True, exist_ok=True)

copy_tasks = []
for cls in ALL_CLASSES:
    for i, src_path in enumerate(balanced_images[cls]):
        suffix = src_path.suffix.lower()
        if suffix not in [".jpg", ".jpeg", ".png"]:
            suffix = ".jpg"
        dst_path = LOCAL_CACHE / cls / f"{cls}_{i:05d}{suffix}"
        copy_tasks.append((src_path, dst_path))

print(f"\n📁 Copying {len(copy_tasks)} files...")
with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(executor.map(copy_file, copy_tasks), total=len(copy_tasks), desc="Copying"))
print(f"✅ Done! {sum(results)} files copied")

In [ ]:
#@title 🔀 **Step 5: Create Train/Val/Test Splits** { display-mode: "form" }

print("🔀 Creating train/val/test splits...")

all_images = []
all_labels = []
class_to_idx = {cls: idx for idx, cls in enumerate(ALL_CLASSES)}

for cls in ALL_CLASSES:
    cls_dir = LOCAL_CACHE / cls
    if not cls_dir.exists():
        continue
    for img_file in cls_dir.glob("*"):
        if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
            all_images.append(str(img_file))
            all_labels.append(class_to_idx[cls])

all_images = np.array(all_images)
all_labels = np.array(all_labels)

train_val_imgs, test_imgs, train_val_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=TEST_SPLIT, stratify=all_labels, random_state=42
)

val_ratio = VALIDATION_SPLIT / (1 - TEST_SPLIT)
train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    train_val_imgs, train_val_labels, test_size=val_ratio, stratify=train_val_labels, random_state=42
)

NUM_CLASSES = len(ALL_CLASSES)
print(f"   ✅ Train: {len(train_imgs)} images")
print(f"   ✅ Val:   {len(val_imgs)} images")
print(f"   ✅ Test:  {len(test_imgs)} images")
print(f"\n📚 Classes ({NUM_CLASSES}): {ALL_CLASSES}")

In [ ]:
#@title ⚡ **Step 6: Create Data Pipeline** { display-mode: "form" }

AUTOTUNE = tf.data.AUTOTUNE

def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_saturation(image, 0.8, 1.2)
    k = tf.random.uniform([], 0, 4, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    return image

def parse_image(file_path, label, training=True):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    if training:
        image = augment(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

def create_dataset(images, labels, training=True):
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    if training:
        ds = ds.shuffle(buffer_size=len(images))
    ds = ds.map(lambda x, y: parse_image(x, y, training), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    if training:
        ds = ds.cache()
    return ds

print("⚡ Creating data pipelines...")
train_ds = create_dataset(train_imgs, train_labels, training=True)
val_ds = create_dataset(val_imgs, val_labels, training=False)
test_ds = create_dataset(test_imgs, test_labels, training=False)

print(f"   ✅ Train batches: {len(train_imgs) // BATCH_SIZE}")
print(f"   ✅ Val batches:   {len(val_imgs) // BATCH_SIZE}")

In [ ]:
#@title 🏗️ **Step 7: Build Model** { display-mode: "form" }

print("🏗️ Building EfficientNetV2-S model...")

base_model = tf.keras.applications.EfficientNetV2S(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
    pooling=None
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(DROPOUT_RATE / 2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

print(f"   ✅ Total params: {model.count_params():,}")
print(f"   ✅ Trainable: {sum(tf.keras.backend.count_params(w) for w in model.trainable_weights):,}")

In [ ]:
#@title ⚖️ **Step 8: Class Weights & Callbacks** { display-mode: "form" }

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))

print("⚖️ Class weights:")
for cls_name, weight in zip(ALL_CLASSES, class_weights):
    print(f"   {cls_name:15s}: {weight:.3f}")

# Callbacks
CHECKPOINT_DIR = OUTPUT_DIR / "checkpoints"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

class TqdmCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs, phase_name="Training"):
        super().__init__()
        self.epochs = epochs
        self.phase_name = phase_name
        self.epoch_bar = None
    def on_train_begin(self, logs=None):
        self.epoch_bar = tqdm(total=self.epochs, desc=f"🚀 {self.phase_name}", unit="epoch")
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch_bar.set_postfix({'acc': f"{logs.get('accuracy', 0):.2%}", 'val_acc': f"{logs.get('val_accuracy', 0):.2%}"})
        self.epoch_bar.update(1)
    def on_train_end(self, logs=None):
        self.epoch_bar.close()

callbacks_base = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=EARLY_STOPPING_PATIENCE, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(str(CHECKPOINT_DIR / "best_model.keras"), monitor='val_accuracy', save_best_only=True, verbose=0)
]

print("\n🎯 Callbacks configured")

In [ ]:
#@title 🚀 **Step 9: Phase 1 - Train Head** { display-mode: "form" }

print("="*60)
print("🚀 PHASE 1: Training Classification Head (Backbone Frozen)")
print("="*60)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE1,
    callbacks=callbacks_base + [TqdmCallback(EPOCHS_PHASE1, "Phase 1: Head")],
    class_weight=class_weight_dict,
    verbose=0
)

print(f"\n✅ Phase 1 Complete! Best val_accuracy: {max(history_phase1.history['val_accuracy']):.4f}")

In [ ]:
#@title 🔥 **Step 10: Phase 2 - Unfreeze Top 30%** { display-mode: "form" }

print("="*60)
print("🔥 PHASE 2: Training Top 30% of Backbone")
print("="*60)

base_model.trainable = True
num_layers = len(base_model.layers)
freeze_until = int(num_layers * 0.7)

for layer in base_model.layers[:freeze_until]:
    layer.trainable = False

print(f"   📊 Total layers: {num_layers}")
print(f"   🔒 Frozen: {freeze_until}")
print(f"   🔥 Trainable: {num_layers - freeze_until}")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE2,
    callbacks=callbacks_base + [TqdmCallback(EPOCHS_PHASE2, "Phase 2: Top 30%")],
    class_weight=class_weight_dict,
    verbose=0
)

print(f"\n✅ Phase 2 Complete! Best val_accuracy: {max(history_phase2.history['val_accuracy']):.4f}")

In [ ]:
#@title 🎓 **Step 11: Phase 3 - Full Fine-tuning** { display-mode: "form" }

print("="*60)
print("🎓 PHASE 3: Full Fine-tuning (All Layers Trainable)")
print("="*60)

for layer in base_model.layers:
    layer.trainable = True

print(f"   🔥 All {len(base_model.layers)} layers now trainable")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR / 5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')]
)

history_phase3 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_PHASE3,
    callbacks=callbacks_base + [TqdmCallback(EPOCHS_PHASE3, "Phase 3: Full")],
    class_weight=class_weight_dict,
    verbose=0
)

print(f"\n✅ Phase 3 Complete! Best val_accuracy: {max(history_phase3.history['val_accuracy']):.4f}")

In [ ]:
#@title 📊 **Step 12: Training Visualization** { display-mode: "form" }

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
colors = ['#2ecc71', '#3498db', '#9b59b6']
histories = [history_phase1, history_phase2, history_phase3]
names = ['Phase 1', 'Phase 2', 'Phase 3']

epoch_offset = 0
for hist, name, color in zip(histories, names, colors):
    epochs = range(epoch_offset, epoch_offset + len(hist.history['loss']))
    axes[0].plot(epochs, hist.history['loss'], color=color, linestyle='-', label=f'{name} Train')
    axes[0].plot(epochs, hist.history['val_loss'], color=color, linestyle='--', label=f'{name} Val')
    axes[1].plot(epochs, hist.history['accuracy'], color=color, linestyle='-', label=f'{name} Train')
    axes[1].plot(epochs, hist.history['val_accuracy'], color=color, linestyle='--', label=f'{name} Val')
    axes[2].plot(epochs, hist.history['top3_accuracy'], color=color, linestyle='-', label=f'{name} Train')
    axes[2].plot(epochs, hist.history['val_top3_accuracy'], color=color, linestyle='--', label=f'{name} Val')
    epoch_offset += len(hist.history['loss'])

axes[0].set_title('Loss'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
axes[1].set_title('Accuracy'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
axes[2].set_title('Top-3 Accuracy'); axes[2].legend(); axes[2].grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'training_history.png'), dpi=150)
plt.show()

In [ ]:
#@title 🧪 **Step 13: Model Evaluation** { display-mode: "form" }

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("="*60)
print("🧪 FINAL EVALUATION ON TEST SET")
print("="*60)

test_loss, test_acc, test_top3 = model.evaluate(test_ds, verbose=1)
print(f"\n📊 Test Results:")
print(f"   Loss:          {test_loss:.4f}")
print(f"   Accuracy:      {test_acc:.2%}")
print(f"   Top-3 Accuracy: {test_top3:.2%}")

y_true, y_pred = [], []
for images, labels in tqdm(test_ds, desc="🔮 Predicting"):
    predictions = model.predict(images, verbose=0)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(predictions, axis=1))

print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=ALL_CLASSES, digits=3))

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=ALL_CLASSES, yticklabels=ALL_CLASSES)
plt.title('Confusion Matrix')
plt.xlabel('Predicted'); plt.ylabel('True')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(str(OUTPUT_DIR / 'confusion_matrix.png'), dpi=150)
plt.show()

In [ ]:
#@title 💾 **Step 14: Save Model** { display-mode: "form" }

import json

print("="*60)
print("💾 SAVING FINAL MODEL")
print("="*60)

# Save model
final_model_path = OUTPUT_DIR / "plantvillage-npk-v3.h5"
model.save(str(final_model_path))
print(f"   ✅ Saved: {final_model_path}")

# Save class names
with open(OUTPUT_DIR / "class_names.txt", 'w') as f:
    for cls in ALL_CLASSES:
        f.write(cls + '\n')
print(f"   ✅ Saved: class_names.txt")

# Save config
config = {
    'model': 'EfficientNetV2-S',
    'img_size': IMG_SIZE,
    'num_classes': NUM_CLASSES,
    'classes': ALL_CLASSES,
    'test_accuracy': float(test_acc),
    'test_top3_accuracy': float(test_top3)
}
with open(OUTPUT_DIR / "model_config.json", 'w') as f:
    json.dump(config, f, indent=2)
print(f"   ✅ Saved: model_config.json")

print("\n🎉 Training complete!")

In [ ]:
#@title 📥 **Step 15: Download Model** { display-mode: "form" }
#@markdown Download the trained model to your computer

from google.colab import files

print("📥 Downloading trained model...")
print("   (This will trigger a browser download)")

# Download model
files.download(str(OUTPUT_DIR / "plantvillage-npk-v3.h5"))
files.download(str(OUTPUT_DIR / "class_names.txt"))
files.download(str(OUTPUT_DIR / "model_config.json"))

print("\n✅ Files downloaded!")
print("\n📋 Next steps:")
print("   1. Copy plantvillage-npk-v3.h5 to backend/ml/models/")
print("   2. Update MODEL_PATH in your backend config")
print("   3. Restart the backend server")
print("   4. Test with the app!")